In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms 
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

# Define paths
DATA_DIR = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)'
DATA_DIR1='/kaggle/input/new-plant-diseases-dataset'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'valid')
TEST_DIR = os.path.join(DATA_DIR1, 'test')

# Get class names from directories
class_names = sorted(os.listdir(TRAIN_DIR))
num_classes = len(class_names)

# Define custom dataset class
class PlantDiseaseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        
        self.image_paths = []
        self.labels = []
        
        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            class_idx = self.class_to_idx[class_name]
            
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.image_paths.append(img_path)
                self.labels.append(class_idx)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

# Define data transformations
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define datasets and dataloaders
train_dataset = PlantDiseaseDataset(TRAIN_DIR, transform=train_transforms)
valid_dataset = PlantDiseaseDataset(VALID_DIR, transform=test_transforms)
test_dataset = PlantDiseaseDataset(TEST_DIR, transform=test_transforms)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

print(f"Number of classes: {num_classes}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(valid_dataset)}")
print(f"Testing samples: {len(test_dataset)}")

# Custom CNN Model
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.bn5 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = F.relu(self.bn4(self.conv4(x)))
        x = x.view(-1, 256 * 14 * 14)
        x = self.dropout(x)
        x = F.relu(self.bn5(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Transfer Learning with ResNet50
class TransferLearningModel(nn.Module):
    def __init__(self, num_classes):
        super(TransferLearningModel, self).__init__()
        self.resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        return self.resnet(x)

# Training and evaluation functions
def train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler, num_epochs=10, device='cuda'):
    from tqdm.auto import tqdm
    
    model.to(device)
    best_val_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    # Create epoch progress bar
    epoch_pbar = tqdm(range(num_epochs), desc="Training Progress", position=0)
    
    for epoch in epoch_pbar:
        # Training phase
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0
        
        # Create batch progress bar for training
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", 
                          leave=False, position=1)
        
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            # Update training progress bar
            batch_loss = loss.item()
            batch_acc = (predicted == labels).sum().item() / labels.size(0)
            train_pbar.set_postfix({'loss': f'{batch_loss:.4f}', 'acc': f'{batch_acc:.4f}'})
        
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        # Create batch progress bar for validation
        val_pbar = tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Valid]", 
                        leave=False, position=1)
        
        with torch.no_grad():
            for inputs, labels in val_pbar:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
                # Update validation progress bar
                batch_loss = loss.item()
                batch_acc = (predicted == labels).sum().item() / labels.size(0)
                val_pbar.set_postfix({'loss': f'{batch_loss:.4f}', 'acc': f'{batch_acc:.4f}'})
                
        val_loss = val_loss / len(valid_loader.dataset)
        val_acc = val_correct / val_total
        
        # Update learning rate
        scheduler.step(val_loss)
        
        # Record history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Update epoch progress bar with overall metrics
        epoch_pbar.set_postfix({
            'train_loss': f'{train_loss:.4f}', 
            'train_acc': f'{train_acc:.4f}',
            'val_loss': f'{val_loss:.4f}', 
            'val_acc': f'{val_acc:.4f}'
        })
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            print(f'\nSaved new best model with validation accuracy: {val_acc:.4f}')
    
    return model, history
def evaluate_model(model, test_loader, device='cuda'):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=class_names)
    conf_matrix = confusion_matrix(all_labels, all_preds)
    
    print(f'Test Accuracy: {accuracy:.4f}')
    print('\nClassification Report:')
    print(report)
    
    # Plot confusion matrix
    plt.figure(figsize=(15, 15))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    
    return accuracy, report, conf_matrix

# Main execution block
if __name__ == "__main__":
    # Choose which model to use
    use_transfer_learning = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    if use_transfer_learning:
        model = TransferLearningModel(num_classes)
        print("Using ResNet50 transfer learning model")
    else:
        model = CustomCNN(num_classes)
        print("Using custom CNN model")
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1)
    
    # Train the model
    num_epochs = 10
    model, history = train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler, num_epochs, device)
    
    # Load best model
    model.load_state_dict(torch.load('best_model.pth'))
    
    # Evaluate on test set
    accuracy, report, conf_matrix = evaluate_model(model, test_loader, device)
    
    # Save the class names for use in the web app
    import json
    with open('class_names.json', 'w') as f:
        json.dump(class_names, f)
    
    # Save the model in TorchScript format for deployment
    model.eval()
    example = torch.rand(1, 3, 224, 224).to(device)
    traced_script_module = torch.jit.trace(model, example)
    traced_script_module.save("plant_disease_model.pt")
    
    print("Model trained, evaluated, and saved for deployment!")

In [ ]:
# # When exporting your model from Python
# import torch
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torchvision
# import torchvision.transforms as transforms 
# from torchvision.models import resnet50, ResNet50_Weights
# from torch.utils.data import DataLoader, Dataset
# import os
# from PIL import Image
# import numpy as np
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# import matplotlib.pyplot as plt
# import seaborn as sns
# import tqdm
# class TransferLearningModel(nn.Module):
#     def __init__(self, num_classes):
#         super(TransferLearningModel, self).__init__()
#         self.resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
#         in_features = self.resnet.fc.in_features
#         self.resnet.fc = nn.Sequential(
#             nn.Linear(in_features, 512),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Linear(512, num_classes)
#         )
        
#     def forward(self, x):
#         return self.resnet(x)
# model = TransferLearningModel(38)
# # Load your trained model
# # model = YourModel()
# model.load_state_dict(torch.load('model_output/best_model.pth', map_location=torch.device('cpu')))


# # Set to evaluation mode
# model.eval()

# # IMPORTANT: Move model to CPU before exporting
# model = model.to('cpu')

# # Create a sample input tensor (for tracing)
# example_input = torch.rand(1, 3, 224, 224)

# # Export using scripting or tracing
# # Option 1: Scripting
# # scripted_model = torch.jit.script(model)
# # scripted_model.save("plant_disease_model.pt")

# # Option 2: Tracing (often better for mobile)
# traced_model = torch.jit.trace(model, example_input)
# traced_model.save("plant_disease_model1.pt")

# # Verify it works with CPU
# test_output = traced_model(example_input)
# print("Model successfully exported and verified on CPU!")

Model successfully exported and verified on CPU!
